In [2]:
import os
import sys
import gmsh
import numpy as np
import pandas as pd
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.tri import Triangulation
from scipy.constants import mu_0, epsilon_0, speed_of_light
from scipy.special import jvp, hankel2, h2vp, jv

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import graph_results

# Constants

In [3]:
OMEGA = 2 * np.pi * 3E8
K0 = OMEGA * np.sqrt(mu_0 * epsilon_0)
WAVELENGTH = 2 * np.pi / K0
OMEGA = {'a': {'h': WAVELENGTH/20, 'L': WAVELENGTH*0.5, 'ra': WAVELENGTH*0.5, 'x0': 1.0*WAVELENGTH},
         'b': {'h': WAVELENGTH/20, 'L': WAVELENGTH*0.5, 'ra': WAVELENGTH*0.5, 'x0': 1.2*WAVELENGTH},
         'c': {'h': WAVELENGTH/20, 'L': WAVELENGTH*0.4, 'ra': WAVELENGTH*0.5, 'x0': 1.0*WAVELENGTH}}

# `create_domain()`

In [4]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, OMEGA_KEY, auto_save=True, view_mesh=False):
    # Define os parâmetros de entrada
    type, order = FINITE_ELEMENT

    # Dimensões do domínio 
    h = OMEGA[OMEGA_KEY]['h']     # Tamanho do elemento
    L = OMEGA[OMEGA_KEY]['L']     # Lado do retângulo externo
    ra = OMEGA[OMEGA_KEY]['ra']   # Raio do furo circular
    x0 = OMEGA[OMEGA_KEY]['x0']   # Lado do retângulo interno
    y0 = x0
    
    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("rectangular_pml")
    factory = gmsh.model.occ

    # Criar as superfícies retangulares externa e interna
    rect_outer = factory.addRectangle(-(x0 + L), -(y0 + L), 0, 2 * (x0 + L), 2 * (y0 + L))
    rect_inner = factory.addRectangle(-x0, -y0, 0, 2 * x0, 2 * y0)
    
    disk = factory.addDisk(0, 0, 0, ra, ra)
    outDimTags_plm_omega, _ = factory.cut([(2, rect_outer)], [(2, rect_inner)], removeTool=False)
    outDimTags_fs_omega, _ = factory.cut([(2, rect_inner)], [(2, disk)], removeTool=True)

    # Sincronizar após o corte do retângulo interno
    factory.synchronize()

    # Obter os contornos (curvas, dim=1) de cada superfície
    boundary_pml_ext = gmsh.model.getBoundary(outDimTags_plm_omega, oriented=True, recursive=False)
    boundary_pml_inn = gmsh.model.getBoundary(outDimTags_fs_omega, oriented=True, recursive=False)

    # Exibir os TAGs das curvas associadas a cada contorno
    tagList_scatterer = [-tag[1] for tag in boundary_pml_inn if tag[1] < 0]
    tagList_pml_inn = [tag[1] for tag in boundary_pml_inn if tag[1] > 0]
    tagList_pml_ext = [tag[1] for tag in boundary_pml_ext if tag[1] > 0]

    # Adicionar grupos físicos para curvas (Dim=1)
    for i, CurveTagList in enumerate([tagList_scatterer, tagList_pml_inn, tagList_pml_ext]):
        gmsh.model.addPhysicalGroup(1, CurveTagList, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para superfícies (Dim=2)	    
    for i, SurfaceList in enumerate([outDimTags_plm_omega, outDimTags_fs_omega]):
        SurfaceTagList = [DimTag[1] for DimTag in SurfaceList]
        gmsh.model.addPhysicalGroup(2, SurfaceTagList, tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

    # Definir ordem dos elementos
    gmsh.option.setNumber("Mesh.MeshSizeMax", h)
    gmsh.option.setNumber("Mesh.MeshSizeMin", h)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_pml_domain_{type}{order}.msh"
        gmsh.write(file_path)
        gmsh.finalize()

# `calculate_sx()`

In [5]:
def calculate_sx(x, x0, n):
    """
    Calcula o fator de escala s_x para a PML.

    Parâmetros:
        x (float): Coordenada onde s_x será calculado.
        x0 (float): Limite da região PML.
        k (float): Número de onda (k = 2*pi/lambda).
        sigma_0x (float): Parâmetro máximo sigma_0x.
        n (int): Expoente n da função sigma(x).

    Retorna:
        s_x (complex): Fator de escala s_x.
    """
    R_COEFFICIENT = 1E-4
    SIGMA_0X = -np.log(R_COEFFICIENT) / WAVELENGTH
            
    if abs(x) > x0:
        #sigma_x = SIGMA_0X * np.abs((x - x0) ** n)
        sigma_x = SIGMA_0X * (abs(x) - x0) ** n
    else:
        sigma_x = 0.0
    
    return (1 - 1j * sigma_x / K0)

# `pml_attributes()`

In [6]:
def pml_attributes(x, y, x0, y0, n):
    """
    Define o tensor Lambda(x, y) para a equação de Helmholtz.

    Parâmetros:
        x (float): Coordenada x.
        y (float): Coordenada y.
        s_x_func (function): Função para calcular s_x(x).
        s_y_func (function): Função para calcular s_y(y).

    Retorna:
        Lambda (2D array): Tensor Lambda no ponto (x, y).
    """
    s_x = calculate_sx(x, x0, n)
    s_y = calculate_sx(y, y0, n)
    
    # Define o tensor Lambda(x, y) no ponto (x, y)
    lambda_tensor = np.array([
        [s_y / s_x, 0],
        [0, s_x / s_y]
    ])

    # Define a métrica escalar gamma(x, y) no ponto (x, y)
    gamma = s_x * s_y

    return lambda_tensor, gamma

# `apply_physics()`

In [8]:
def apply_physics(FINITE_ELEMENT, mesh_data, OMEGA_KEY, n):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dimensões do domínio
    x0 = OMEGA[OMEGA_KEY]['x0']   
    y0 = x0
    
    # Desempacotar o tipo de elemento
    type, order = FINITE_ELEMENT

    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in cell_data.items():  
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Calcular o valor de 'a' de acordo com a física do problema
        mur = cell['material']['relative_magnetic_permeability']
        er = cell['material']['relative_electric_permittivity']
        
        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xg_e, yg_e = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            lambda_tensor, gamma = pml_attributes(xg_e, yg_e, x0, y0, n)
            cell['stiffness_term'].append(lambda_tensor)
            cell['mass_term'].append(gamma)           
        
            # Adicionar a nova chave 'source' ao dicionário da célula
            cell['source'].append(0.0)

        # Dictionary with boundary nodes
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    # Adiciona os potenciais de Dirichlet
    for node in nodes_data.values():
        if node['bc']['type'] == 'Dirichlet' and node['bc']['value'] is None:
            # Coordenadas cartesianas do nó
            x, _ = node['xg']  
                        
            # Atualizar o valor de Dirichlet para o nó
            node['bc']['value'] = -np.exp(-1j * K0 * x)
    
    return mesh_data

# `ez_at_node()`

In [9]:
def ez_at_node(node, OMEGA_KEY, series_terms=40):
    ez = 0    
    ra = OMEGA[OMEGA_KEY]['ra']
    
    # Converter coordenadas cartesianas para coordenadas polares
    x, y = node['xg']
    rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)
    for n in range(0, series_terms):
        e_n = 2 if n != 0 else 1
        Jn_ka = jv(n, K0 * ra)
        Hn2_ka = hankel2(n, K0 * ra)
        Hn2_kp = hankel2(n, K0 * rho)
        ez += (-1j) ** n * e_n * (Jn_ka / Hn2_ka) * Hn2_kp * np.cos(n * phi)
    return ez

# `ez_at_omega()`

In [10]:
def ez_at_omega(mesh_data, OMEGA_KEY):
    ez = {}
    for key, node in mesh_data['nodes'].items():
        ez[key] = ez_at_node(node, OMEGA_KEY)
    return ez

# `ezh_at_contour()`

In [11]:
def ez_at_contour(mesh_data, contour='BGT'):
    ez = {}
    for key, node in mesh_data['nodes'].items():
        if node['bc']['type'] == contour:
            # Coordenadas cartesianas do nó
            x, y = node['xg']
            
            # Converter coordenadas cartesianas para coordenadas polares
            rho, phi = np.sqrt(x**2 + y**2), np.arctan2(y, x)

            # Converter phi para o intervalo [0, 2*pi]
            if phi < 0:
                phi = 2 * np.pi + phi

            # Analytical solution
            ez[key] = {
                'phi': phi,
                'ez': ez_at_node(node)
            }
    return ez

# `calculate_error()`

In [12]:
def calculate_error(u_exact, u_approx):
    """
    Calcula o erro entre a solução exata e a solução aproximada.

    Parâmetros:
    - u_exact: Array com os valores da solução exata u_ex.
    - u_approx: Array com os valores da solução aproximada u_h.

    Retorna:
    - erro: Valor do erro calculado.
    """
    # Verificar se os arrays têm o mesmo tamanho
    if len(u_exact) != len(u_approx):
        raise ValueError("Os arrays u_exact e u_approx devem ter o mesmo tamanho.")

    # Calcular a soma dos quadrados das diferenças
    N = len(u_exact)
    error = np.sqrt(np.sum((u_exact - u_approx)**2) / N)
    return np.abs(error)

# `plot_coordinates()`

In [13]:
def plot_coordinates(mesh_data):
    pec_coords = []  # Coordenadas de nós com tipo PEC
    pml_coords = []  # Coordenadas de nós com tipo BGT

    for node in mesh_data['nodes'].values():
        x, y = node['xg']
        if node['bc']['type'] == 'Dirichlet':
            pec_coords.append((x, y))
        elif node['bc']['type'] == 'PML':
            pml_coords.append((x, y))

    # Conversão para listas separadas de x e y
    pec_x, pec_y = zip(*pec_coords) if pec_coords else ([], [])
    pml_x, pml_y = zip(*pml_coords) if pml_coords else ([], [])

    # Visualização
    plt.figure(figsize=(8, 8))
    plt.scatter(pec_x, pec_y, color='blue', label='PEC', marker='o', s=10)
    plt.scatter(pml_x, pml_y, color='red', label='PML', marker='x', s=10)
    plt.axhline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.axvline(0, color='gray', linewidth=0.5, linestyle='--')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Visualização de Nós por Tipo de Condição de Contorno")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

# `plot_ez_domain()`

In [14]:
def plot_ez_domain(FINITE_ELEMENT, mesh_data, u_dict, type='abs'):
    # Conversão do dicionário de soluções para lista
    if type == 'real':
        u = np.real(list(u_dict.values()))
    elif type == 'imag':
        u = np.imag(list(u_dict.values()))
    elif type == 'abs':
        u = np.abs(list(u_dict.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")

    # Extração de informações sobre o elemento finito
    type, order = FINITE_ELEMENT

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn = graph_results.structured_data(mesh_data)

    # Verificação do tipo e ordem do elemento
    if type == "Triangle":
        triangulation = Triangulation(xg, yg, conn)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, u, cmap='viridis')
    plt.colorbar(label=r'$u(x, y)$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()

    plt.show()
    # # Salvando o gráfico
    # filepath = graph_results.get_dir(f"pos_processing/pictures/analytical_solution.svg")
    # plt.savefig(filepath, format="svg")
    # plt.close()
    # print(f"Arquivo salvo em: {filepath}")

# `plot_ez_physical_domain()`

In [ ]:
def plot_ez_physical_domain(mesh_data, ez_h):
    # Filtrar elementos fora da PML
    physical_elements = [
        cell for cell in mesh_data['cell'].values()
        if cell['material']['name'] != 'PML'
    ]

    # Obter conectividade e coordenadas dos nós físicos
    physical_conn = [elem['conn'] for elem in physical_elements]
    physical_nodes = set(node for conn in physical_conn for node in conn)

    # Extrair coordenadas dos nós físicos
    xg_physical = [mesh_data['nodes'][node]['xg'][0] for node in physical_nodes]
    yg_physical = [mesh_data['nodes'][node]['xg'][1] for node in physical_nodes]

    # Recriar conectividade com índices ajustados
    node_index_map = {node: idx for idx, node in enumerate(sorted(physical_nodes))}
    adjusted_conn = [
        [node_index_map[node] for node in conn] for conn in physical_conn
    ]

    # Crie a triangulação
    triangulation = Triangulation(
        [xg_physical[node_index_map[node]] for node in physical_nodes],
        [yg_physical[node_index_map[node]] for node in physical_nodes],
        adjusted_conn
    )

    # Obter valores de solução apenas para os nós físicos
    u_physical = [ez_h[node] for node in physical_nodes]

    # Convertendo o dicionário de solução
    u = np.abs(list(u_physical))  # Pode ser 'real', 'imag' ou 'abs'

    # Plotando
    plt.figure(figsize=(7, 5))
    plt.tricontourf(triangulation, u, cmap='viridis')
    plt.colorbar(label=r'$u(x, y)$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.show()

# `plot_ez_profile()`

In [15]:
def plot_ez_profile(ez, ez_h):
    phi = [value['phi'] for value in ez.values()]
    ez = [value['ez'] for value in ez.values()]
    ez_h = [-value for value in ez_h.values()]

    # Parte Real de Ez
    plt.figure(figsize=(8, 6))
    plt.plot(np.degrees(phi), np.real(ez), 
            label='Analytical', color='blue')
    plt.plot(np.degrees(phi), np.real(ez_h), 
            label='FEM (BGT-2)', color='red', linestyle='--')
    plt.title(r"Scatterer Electric Field, $E_z$, at domain boundary $R = 1.5 \lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Re(E_z)$ $[V/m]$")
    plt.legend()
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)
    plt.show()

    # Parte Imaginária de Ez
    plt.figure(figsize=(8, 6))
    plt.plot(np.degrees(phi), np.imag(ez), 
            label='Analytical', color='blue')
    plt.plot(np.degrees(phi), np.imag(ez_h), 
            label='FEM (BGT-2)', color='red', linestyle='--')
    plt.title(r"Scatterer Electric Field, $E_z$, at domain boundary $R = 1.5 \lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Im(E_z)$ $[V/m]$")
    plt.legend()
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)
    plt.show()

    # Módulo de Ez
    plt.figure(figsize=(8, 6))
    plt.plot(np.degrees(phi), np.abs(ez), 
            label='Analytical', color='blue')
    plt.plot(np.degrees(phi), np.abs(ez_h), 
            label='FEM (BGT-2)', color='red', linestyle='--')
    plt.title(r"Scatterer Electric Field, $E_z$, at domain boundary $R = 1.5 \lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$|E_z|$ $[V/m]$")
    plt.legend()
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)
    plt.show()

# `plot_ez_multi_profile()`

In [16]:
def plot_ez_multi_profile(ez_results, r_domain):
    phi = [value['phi'] for value in ez_results['ez'].values()]
    ez = [value['ez'] for value in ez_results['ez'].values()]
    
    # Parte Real de Ez
    plt.figure(figsize=(8, 6))
    # Resultado Numérico
    plt.plot(np.degrees(phi), np.real(ez), 
            label='Analytical', color='black')
    # Plotar os resultados para as diferentes condições de contorno
    for bgt in ['0', '1', '2']:
        ezh_bgt = [-value for value in ez_results[bgt].values()]
        plt.plot(np.degrees(phi), np.real(ezh_bgt), 
                label=f'BGT-{bgt}', linestyle='--')
    plt.title(fr"Scatterer Electric Field, $E_z$, at domain boundary $R = {r_domain}\lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Re(E_z)$ $[V/m]$")
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)    
    plt.legend()
    plt.show()

    # Parte Imaginária de Ez
    plt.figure(figsize=(8, 6))
    # Resultado Numérico
    plt.plot(np.degrees(phi), np.imag(ez), 
            label='Analytical', color='black')
    # Plotar os resultados para as diferentes condições de contorno
    for bgt in ['0', '1', '2']:
        ezh_bgt = [-value for value in ez_results[bgt].values()]
        plt.plot(np.degrees(phi), np.imag(ezh_bgt), 
                label=f'BGT-{bgt}', linestyle='--')
    plt.title(fr"Scatterer Electric Field, $E_z$, at domain boundary $R = {r_domain}\lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$Im(E_z)$ $[V/m]$")
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)    
    plt.legend()
    plt.show()

    # Módulo de Ez
    plt.figure(figsize=(8, 6))
    # Resultado Numérico
    plt.plot(np.degrees(phi), np.abs(ez), 
            label='Analytical', color='black')
    # Plotar os resultados para as diferentes condições de contorno
    for bgt in ['0', '1', '2']:
        ezh_bgt = [-value for value in ez_results[bgt].values()]
        plt.plot(np.degrees(phi), np.abs(ezh_bgt), 
                label=f'BGT-{bgt}', linestyle='--')
    plt.title(fr"Scatterer Electric Field, $E_z$, at domain boundary $R = {r_domain}\lambda$")
    plt.xlabel(r"$\phi$ $[deg]$")
    plt.ylabel(r"$|E_z|$ $[V/m]$")
    plt.xticks(np.arange(0, 361, 60))
    plt.grid(True)
    plt.xlim(0, 360)    
    plt.legend()
    plt.show()

# `resume_info()`

In [17]:
def resume_info(ez, ez_h):
    data = []
    for node, ez_value in ez.items():
        ezh_value = ez_h[node]
        abs_ez = abs(ez_value)
        abs_ezh = abs(ezh_value)

        data.append({
            "Node": node,
            "ez": ez_value,
            "ez_h": ezh_value,
            "|ez|": abs_ez,
            "|ez_h|": abs_ezh,
            "Error (%)": (abs_ez - abs_ezh) / abs_ez * 100
        })
    return pd.DataFrame(data)

# `resume_physical_info()`

In [ ]:
def resume_physical_info(mesh_data, ez, ez_h):
    # Identificar nós pertencentes apenas ao domínio físico
    physical_nodes = {
        node for cell in mesh_data['cell'].values()
        if cell['material']['name'] != 'PML'
        for node in cell['conn']
    }

    # Coletar informações apenas para nós físicos
    data = []
    for node in physical_nodes:
        ez_value = ez.get(node, None)
        ezh_value = ez_h.get(node, None)

        # Ignorar nós que não têm valores em ez ou ez_h
        if ez_value is None or ezh_value is None:
            continue

        abs_ez = abs(ez_value)
        abs_ezh = abs(ezh_value)

        # Evitar divisão por zero em erros relativos
        error_percent = (
            (abs_ez - abs_ezh) / abs_ez * 100 if abs_ez != 0 else None
        )

        data.append({
            "Node": node,
            "ez": ez_value,
            "ez_h": ezh_value,
            "|ez|": abs_ez,
            "|ez_h|": abs_ezh,
            "Error (%)": error_percent
        })

    return pd.DataFrame(data)

# `mesh_data_to_table()`

In [18]:
def cell_data_to_table(mesh_data):
    data = []
    for key, cell in mesh_data['cell'].items():
        row = {
            "Cell": key,
            # "Connections": cell['conn'],
            # "Material Tag": cell['material']['tag'],
            # "Material Name": cell['material']['name'],
            # "Rel. Magnetic Permeability": cell['material']['relative_magnetic_permeability'],
            # "Rel. Electric Permittivity": cell['material']['relative_electric_permittivity'],
            # "Source Type": cell['source']['type'],
            # "Source Value": cell['source']['value'],
            "Stiffness Matrix": cell['stiffness_a_value'],
            "Mass Value": cell['mass_a_value'],
            # "ABC Type": cell['abc']['type'],
            # "ABC Connections": cell['abc']['conn_idx']
        }
        data.append(row)

    return pd.DataFrame(data)

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.